# 镶嵌栅格

In [1]:
import os
import arcpy
from arcpy import env
from arcpy.sa import *
import re

In [2]:
raster_path = r"F:\GEEDOWNLOAD\sentinel2\GL"

In [3]:
raster_list = []
for roots,dirs,files in os.walk(raster_path):
    for one_file in files:
        if one_file.endswith(".tif") or one_file.endswith(".BIL"):
            raster_list.append(os.path.join(roots,one_file))
# 使用正则表达式匹配下划线和点号之间的数字
def find_number(path):
    match = re.findall(r'_(\d+)\.', path)
    return str(match[0]).zfill(3)
raster_list = sorted(raster_list,key=find_number)

In [4]:
raster_list,len(raster_list)

(['F:\\GEEDOWNLOAD\\sentinel2\\GL\\Sentinel-2_scl_1.tif',
  'F:\\GEEDOWNLOAD\\sentinel2\\GL\\Sentinel-2_scl_2.tif',
  'F:\\GEEDOWNLOAD\\sentinel2\\GL\\Sentinel-2_scl_3.tif',
  'F:\\GEEDOWNLOAD\\sentinel2\\GL\\Sentinel-2_scl_4.tif',
  'F:\\GEEDOWNLOAD\\sentinel2\\GL\\Sentinel-2_scl_5.tif',
  'F:\\GEEDOWNLOAD\\sentinel2\\GL\\Sentinel-2_scl_6.tif',
  'F:\\GEEDOWNLOAD\\sentinel2\\GL\\Sentinel-2_scl_7.tif',
  'F:\\GEEDOWNLOAD\\sentinel2\\GL\\Sentinel-2_scl_8.tif',
  'F:\\GEEDOWNLOAD\\sentinel2\\GL\\Sentinel-2_scl_9.tif',
  'F:\\GEEDOWNLOAD\\sentinel2\\GL\\Sentinel-2_scl_10.tif',
  'F:\\GEEDOWNLOAD\\sentinel2\\GL\\Sentinel-2_scl_11.tif',
  'F:\\GEEDOWNLOAD\\sentinel2\\GL\\Sentinel-2_scl_12.tif',
  'F:\\GEEDOWNLOAD\\sentinel2\\GL\\Sentinel-2_scl_13.tif',
  'F:\\GEEDOWNLOAD\\sentinel2\\GL\\Sentinel-2_scl_14.tif',
  'F:\\GEEDOWNLOAD\\sentinel2\\GL\\Sentinel-2_scl_15.tif',
  'F:\\GEEDOWNLOAD\\sentinel2\\GL\\Sentinel-2_scl_16.tif',
  'F:\\GEEDOWNLOAD\\sentinel2\\GL\\Sentinel-2_scl_17.tif',
  'F:\

In [5]:
# 镶嵌至新栅格
def mosaic_new_raster(raster_list,out_gdb,out_name,pre_refer,bands:int):
    pre_refer_prj = arcpy.Describe(pre_refer).spatialReference
    with arcpy.EnvManager(parallelProcessingFactor="0"):
        arcpy.management.MosaicToNewRaster(raster_list, out_gdb, out_name, pre_refer_prj, "32_BIT_FLOAT", None, bands, "LAST", "FIRST")

In [6]:
out_gdb = r"F:\cache_data\tif_file_sentinel\gl"
out_name = "gl_bands14.tif"
pre_fer = r"D:\ArcGISProjects\workspace\shbyq\feature_raster_file\features_data_gl.gdb\DEM"

In [8]:
# dem
mosaic_new_raster(raster_list,out_gdb,out_name,pre_fer,1)

In [7]:
# sentinel
mosaic_new_raster(raster_list,out_gdb,out_name,pre_fer,14)

In [19]:
# 处理异常值
def Con_calc(raster,error_list):
    # 按焦点统计


    # 识别异常值
    outCon1 = Con((Raster(raster) >error_list[0]) | (Raster(raster) <=error_list[1]), 1, 0)

    # 使用焦点统计来处理异常值
    neighborhood = NbrCircle(3, "CELL")  # 设置邻域大小和形状
    # outFocalStat = FocalStatistics(raster, neighborhood, "MINIMUM")
    outFocalStat = FocalStatistics(raster, neighborhood, "MEAN")

    # 替换异常值
    outRaster = Con(outCon1 == 1, outFocalStat, raster)
    return outRaster
# 取消并行处理
def disable_parallel_processing(func):
    """"装饰器函数,取消并行处理,因为在部分PC中无法进行并行处理"""
    def wrapper(*args, **kwargs):
        with arcpy.EnvManager(parallelProcessingFactor="0"):
            return func(*args, **kwargs)
    return wrapper

# 填洼
@disable_parallel_processing
def fill_dem(dem_raster):
    """参数默认为最佳"""
    fill_raster = Fill(dem_raster,)
    return fill_raster
    

In [12]:
env.workspace = out_gdb

In [16]:
dem_path = r"D:\ArcGISProjects\workspace\shbyq\features_data_dy.gdb\DEM2"
temp = Con_calc(dem_path,[4000,0])
temp.save('DEM3')

In [20]:
# 填洼
fill_dem('DEM3').save('DEM4')